In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
data=pd.read_csv(r"D:\Imarticus\Capstone 2\complaints_processed.csv")
df = data.dropna(axis=0)

In [3]:
texts = df['narrative'].values

In [4]:
le = LabelEncoder()

labels = le.fit_transform(df['product'])

labels = keras.utils.to_categorical(labels)

In [7]:
texts.shape

(162411,)

In [8]:
labels.shape

(162411, 5)

In [19]:
# Split the data into training, validation, and test sets

train_ratio, val_ratio, test_ratio = 0.7, 0.15, 0.15
total_samples = len(texts)
train_size = int(train_ratio * total_samples)
val_size = int(val_ratio * total_samples)

In [20]:
x_train, y_train = texts[:train_size], labels[:train_size]
x_val, y_val = texts[train_size:train_size+val_size], labels[train_size:train_size+val_size]
x_test, y_test = texts[train_size+val_size:], labels[train_size+val_size:]

In [21]:
# Tokenize the text data

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
vocab_size = len(tokenizer.word_index) + 1

In [22]:
x_train = tokenizer.texts_to_sequences(x_train)
x_val = tokenizer.texts_to_sequences(x_val)
x_test = tokenizer.texts_to_sequences(x_test)

In [23]:
max_sequence_length = 100  # Define your desired sequence length
x_train = pad_sequences(x_train, maxlen=max_sequence_length)
x_val = pad_sequences(x_val, maxlen=max_sequence_length)
x_test = pad_sequences(x_test, maxlen=max_sequence_length)

In [24]:
# Define the RNN model
model = keras.Sequential([
    keras.layers.Embedding(input_dim = vocab_size, output_dim = 100, input_length = max_sequence_length),
    keras.layers.LSTM(128),
    keras.layers.Dense(5, activation='softmax')
])

In [25]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
# Train the model
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=64)

Epoch 1/10
1777/1777 [==============================] - 679s 380ms/step - loss: 0.5761 - accuracy: 0.7989 - val_loss: 0.5176 - val_accuracy: 0.8257
Epoch 2/10
1777/1777 [==============================] - 620s 349ms/step - loss: 0.3763 - accuracy: 0.8740 - val_loss: 0.4778 - val_accuracy: 0.8366
Epoch 3/10
1777/1777 [==============================] - 750s 422ms/step - loss: 0.3171 - accuracy: 0.8931 - val_loss: 0.5119 - val_accuracy: 0.8246
Epoch 4/10
1777/1777 [==============================] - 487s 274ms/step - loss: 0.2711 - accuracy: 0.9086 - val_loss: 0.4767 - val_accuracy: 0.8403
Epoch 5/10
1777/1777 [==============================] - 550s 309ms/step - loss: 0.2311 - accuracy: 0.9217 - val_loss: 0.5111 - val_accuracy: 0.8367
Epoch 6/10
1777/1777 [==============================] - 443s 249ms/step - loss: 0.1947 - accuracy: 0.9343 - val_loss: 0.5694 - val_accuracy: 0.8307
Epoch 7/10
1777/1777 [==============================] - 456s 256ms/step - loss: 0.1633 - accuracy: 0.9451 - val_

In [27]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test accuracy: {accuracy}')

762/762 [==============================] - 36s 47ms/step - loss: 0.7541 - accuracy: 0.8398
Test accuracy: 0.8397570252418518
